To train a model with tensorflow we can simply run a training op a number of times and save a checkpoint of the trained parameters when we're done. This works well for small models that run in a few hours.
Larger models that require days of training, possibly across multiple replicas, need a more robust training process that:
* handles shutdowns and crashes cleanly;
* can be resumed after a shutdown or a crash;
* can be monitored through tensorboard.
To be able to resume training after a crash, the training process must save checkpoints regularly. On restart, it must look for the most recent checkpoint and load it before resuming training.
To be monitored through tensorboard, the training process must run summary ops regularly and append the returned values to an event file. Tensorboard monitors events files and displays graphs reporting training progress over time.
The tf.train.Supervisor provides a set of services that helps implement a robust training process.

Very simple scenario:
1. Create a Supervisor object, passing it to a directory where to save checkpoints and summaries;
2. Ask the supervisor for a session with tf.train.Supervisor.managed_session;
3. Use the session to execute a train op, checking at each step if the supervisor requests that the training stops.

Started Services:
In this scenario, the managed_session() call starts a few services, which run in their own threads, and use the managed session to run ops in our graph.
If our graph contains an integer variable named global_step, the services use its value to measure the number of training steps executed.
1. Checkpointing service: Saves a copy of the graph variables in the logdir;
2. Summary service: Runs all the summary ops and appends their output to an event file in the logdir;
3. Step counter: Counts how many steps have been executed, by looking at changes in the global_step variable;
4. Queue Runners: If any tf.train.QueueRunner were added to the graph, the supervisor launches them in their own threads.

Checking for stop:
The managed_session takes care of catching exceptions raised from the training loop to report them to the supervisor. Exceptions raised in the service threads are also reported to the supervisor which then sets its 'should_stop()' condition to true.

Recovery:
When we restart the program, managed_session restores the graph from the most recent checkpoint and resumes training where it stopped.

In [ ]:
#...create graph...
my_train_op = ...
sv = tf.train.Supervisor(logdir = "/my/training/directory")
with sv.managed_session() as sess:
    for step in xrange(100000):
        if sv.should_stop():
            break;
        sess.run(my_train_op)

Larger model scenario:
Larger model may run out memory when the summary service runs: The summary ops are run in parallel with the main loop running the train op. This can cause memory usage to peak to up to two times the normal use.
For a larger model, we can tell the supervisor to not run the summary services and instead run it ourself in our main training loop. Pass summary_op = None when constructing supervisor.

In [ ]:
#...create graph...
my_train_op = ...
my_summary_op = tf.summary.merge_all()

sv = tf.train.Supervisor(logdir = "/my/training/directory",
                        summary_op = None)
with sv.managed_session() as sess:
    for step in xrange(100000):
        if sv.should_stop():
            break;
        if step % 100 == 0:
            _, summ = sess.run([my_train_op, my_summary_op])
            sv.summary_computed(sess, summ)
        else:
            sess.run(my_train_op)

Pre-trained model scenario:
We can load a pre-trained checkpoint by passing an "init function" to the supervisor. This function is only called if the model needs to be initialized from scratch, not when can be recovered from a checkpoint from the logdir.

In [ ]:
#... create graph...
# Create a saver that restores only the pre-trained variables.
pre_train_saver = tf.train.Saver([pre_train_val1, pre_train_val2])

# Define an init function that loads the pretrained checkpoint
def load_pretrain(sess):
    pre_train_saver.restore(sess, "<path to pre-trained-checkpoint>")
    
# Pass the init function to the supervisor.
# The init function is called _after_ the variables have been initialized by running the init_op.
sv = tf.train.Supervisor(logdir = "/my/training/directory",
                        init_fn = load_pretrain)
with sv.managed_session as sess:
    # Here sess was either initialized from the pre-trained checkpoint or recovered from a checkpoint saved in
    # a previous run of this code.

Running our own customized services:
We can use tf.train.Supervisor.loop method of the supervisor to add our own services. It repeatedly calls a function of our choice on a timer until the supervisor stop condition met, so it plays nicely with the other services.

In [ ]:
# Example: call my_additonal_summaries every 20 mn
def my_additional_summaries(sv, sess):
    #...fetch and write summaries , see below...
    summaries = sess.run(my_additional_summaries_op)
    #This method appends our customized summaries string into the same event log file provided by sv.
    sv.summary_computed(sess, summaries)
sv = tf.train.Supervisor(logdir = "/my/training/directory")
with sv.managed_session as sess:
    #call my_additional_summaries every 1200s
    #passing (sv, sess) as arguments.
    sv.loop(1200, my_additional_summaries, args = (sv, sess))
    #... main train loop ...

Supervisor Reference:
1. Checkpointing: Where and When.
logdir: where to save checkpoints files;
checkpoint_basename: Name of the checkpoints files to create, defaults: 'model.ckpt';
save_model_secs: Number of seconds between two checkpoints;
saver: a tf.train.Saver object to use for checkpointing, if we do not pass one, the sv creates one for us and add ops to save and restore all variables in our model.
2. Summaries: Where and When.
logdir: where to save event log files
save_summaries_secs: Number of seconds between two each run of the summary services;
summary_op: Op to use to fetch the summaries, if not specified the sv use the first op in the tf.GraphKeys.SUMMARY_OP graph collection. If the collection is empty, it creates an op that aggregates all summaries in the graph using tf.summary.merge_all();
global_step: Tensor to use to count the global step. If not specified, it uses the first tensor in the tf.GraphKeys.GLOBAL_STEP graph collection. If it's empty, it looks for a scalar integer variable named global_step in the graph.
3. Model initialization and recovery.
The managed_session() call takes care of initializing or recovering a session. It returns a session with a fully initialized model, ready to run ops. If a checkpoint exists in the logdir, the model is initialized by loading the checkpoint, otherwise it's initialized by calling an init_op and optionally an init function.
init_op: op to run to initialize the model;
init_fn: python function to call to initialize the model;
local_init_op: an additional op to initialize parts of the graph that are not saved in checkpoints such as tables and local variables.It is run before running the ready op the first time, to initialize tables and local variables.
ready_op: op to check if the model is initialized. This op is run before and after loading the checkpoint. First run checks if the model needs to be initialized, second run verifies that the model is fully initialized;